<a href="https://colab.research.google.com/github/hiumar/Quarter2/blob/main/Rag_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
from google.colab import userdata
GEMNI_API_KEY:str=userdata.get('Gemni_Api_Key')

In [1]:
import urllib
import warnings
from pathlib import Path as p
from pprint import pprint

In [5]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 54.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 5.0 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.12
    Uninstalling langchain-0.3.12:
      Successfully uninstalled langchain-0.3.12


In [10]:
from IPython import get_ipython
from IPython.display import display
# %%
import urllib
import warnings
from pathlib import Path as p
from pprint import pprint
# %%
!pip install -U langchain-community
# %%
import pandas as pd
from langchain import PromptTemplate
from langchain.chains.question_answering import load_qa_chain
from langchain.document_loaders import PyPDFLoader # Import PyPDFLoader directly from langchain.document_loaders
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA

warnings.filterwarnings('ignore')
# %%


In [11]:
!pip install pypdf # Install the pypdf package
Loader=PyPDFLoader("/content/NIPS-2017-attention-is-all-you-need-Paper.pdf")

In [14]:
data=Loader.load()
#print(data)
len(data)

11

In [31]:
from langchain.text_splitter import RecursiveCharacterTextSplitter # Import RecursiveCharacterTextSplitter

data=Loader.load()
#print(data)
#len(data)

text_spliter=RecursiveCharacterTextSplitter(chunk_size=10000,chunk_overlap=1000) # Use chunk_size and chunk_overlap
context="\n\n".join(str(item.page_content) for item in data)
texts=text_spliter.split_text(context)

In [29]:
!pip install langchain-google-genai # Install the required package
from langchain_google_genai import GoogleGenerativeAIEmbeddings # Import the GoogleGenerativeAIEmbeddings class
!pip install chromadb


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 605.5/605.5 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 51.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 76.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 90.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 6.4 MB/s eta 0:00:0

In [30]:
from re import search
embedding=GoogleGenerativeAIEmbeddings(model="models/embedding-001",google_api_key=GEMNI_API_KEY)
vector_index=Chroma.from_texts(texts,embedding).as_retriever(search_kwargs={"k":5})


In [33]:
!pip install langchain-google-genai # Install the langchain-google-genai package if not already installed
from langchain_google_genai.chat_models import ChatGoogleGenerativeAI # Import the ChatGoogleGenerativeAI class


In [34]:
llm=ChatGoogleGenerativeAI(model="gemini-2.0-flash-exp",google_api_key=GEMNI_API_KEY,temperature=0.3,convert_system_message_to_human=True)
chain=load_qa_chain(llm,chain_type="stuff")

In [35]:
qa_chain=RetrievalQA.from_chain_type(
    llm,
    retriever=vector_index,
    return_source_documents=True
    )

In [50]:
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer.
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)# Run chain
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vector_index,
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)


In [47]:
QA_CHAIN_PROMPT=PromptTemplate.from_template(template)

In [53]:
question="Describe the multi-head attention in details?"
result=qa_chain({"query":question})
print(result["result"])

Multi-head attention linearly projects queries, keys, and values h times with different learned linear projections to dk, dk, and dv dimensions, respectively. It then performs the attention function in parallel on each of these projected versions, yielding dv-dimensional output values. These are concatenated and projected again, resulting in the final values. Thanks for asking!



In [55]:

question = "Describe Random forest?"
result = qa_chain({"query": question})
print(result["result"])

I don't know, thanks for asking!

